$\large\textbf{The need for the regularization}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import random
from sklearn.datasets import load_digits

random.seed(1000)

### $\textbf{General Code}$

In [ ]:
def evalf(A,x,y,n_feat, n, lamb):
  assert type(A) is np.ndarray and A.shape == (n, n_feat)
  assert type(x) is np.ndarray and x.shape == (n_feat,1)
  assert type(y) is np.ndarray and y.shape == (n,1)
  assert type(n_feat) is int and n_feat >0
  assert type(n) is int and n >0
  assert lamb > 0 

  f = np.matmul(A,x) - y

  return 0.5*(np.linalg.norm(f))**2  + 0.5*lamb*(np.matmul(x.T , x))


def evalg(A,x,y,n_feat, n, lamb):
  assert type(A) is np.ndarray and A.shape == (n, n_feat)
  assert type(x) is np.ndarray and x.shape == (n_feat,1)
  assert type(y) is np.ndarray and y.shape == (n,1)
  assert type(n_feat) is int and n_feat >0
  assert type(n) is int and n >0
  assert lamb >= 0

  d = np.matmul(A,x) - y
  g = np.matmul(A.T , d) + lamb*x

  return g



def evalh(A,x,y,n_feat, n, lamb):
  assert type(A) is np.ndarray and A.shape == (n, n_feat)
  assert type(x) is np.ndarray and x.shape == (n_feat,1)
  assert type(y) is np.ndarray and y.shape == (n,1)
  assert type(n_feat) is int and n_feat >0
  assert type(n) is int and n >0

  h = np.matmul(A.T , A) + lamb*np.identity(n_feat)

  return h


In [ ]:
def compute_steplength_backtracking_scaled_direction(A,x,y, n_feat, n ,lamb  ,gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and x.shape == (n_feat,1)
  assert type(gradf) is np.ndarray and gradf.shape == (n_feat,1)
  assert type(direction) is np.ndarray and direction.shape == (n_feat, n_feat)
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 

  alpha = alpha_start
  p = -gradf

  while (evalf(A, x + alpha*np.matmul(direction,p), y, n_feat, n,lamb) > (evalf(A,x,y,n_feat,n,lamb) + gamma * alpha * np.matmul(gradf.T, np.matmul(direction, p))) ):
    alpha = alpha*rho
 
  return alpha


#line search type 
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1



def find_minimizer_Newtonmethod(A, start_x, y, n_feat,  n,lamb, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(A) is np.ndarray and A.shape == (n, n_feat)
  assert type(start_x) is np.ndarray and start_x.shape == (n_feat,1)
  assert type(y) is np.ndarray and y.shape == (n,1)
  assert type(n_feat) is int and n_feat >0
  assert type(n) is int and n >0
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(A,x,y,n_feat,n,lamb)
  

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  x_k = []
  
  #print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol):

    d = np.linalg.inv(evalh(A,x,y,n_feat,n,lamb))

    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(A,x,y,n_feat,n,lamb,g_x, d , alpha_start,rho, gamma)
 
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1.0
 
    else:  
      raise ValueError('Line search type unknown. Please check!')
 
    x = np.subtract(x, step_length * np.matmul(d,g_x)) 
    k += 1 
    x_k.append(x)
    g_x = evalg(A,x,y,n_feat,n,lamb)
  
  return x, k, x_k



def find_minimizer_BFGS(A, start_x, y, n_feat, n,lamb, tol,line_search_type,*args):
  assert type(A) is np.ndarray and A.shape == (n, n_feat)
  assert type(start_x) is np.ndarray and start_x.shape == (n_feat,1)
  assert type(y) is np.ndarray and y.shape == (n,1)
  assert type(n_feat) is int and n_feat >0
  assert type(n) is int and n >0
  assert type(tol) is float and tol>=0 


  x = start_x
  k = 0
  g_new = evalg(A,x,y,n_feat,n,lamb)
  B = np.identity(n_feat)
  x_k = []  

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  while (np.linalg.norm(g_new) > tol):

    d = B

    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(A,x,y,n_feat,n,lamb ,g_new, d , alpha_start,rho, gamma)
 
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1.0
 
    else:  
      raise ValueError('Line search type unknown. Please check!')
      
    g_old = evalg(A,x,y,n_feat,n,lamb)
    p = np.matmul(-1*B, g_old)
    x = x + step_length * p
    s = step_length * p
    g_new = evalg(A,x,y,n_feat,n,lamb)
    y_k = g_new - g_old

    mu = 1 / (np.matmul(y_k.T,s))
    term1 = np.identity(n_feat) - mu * np.matmul(s,y_k.T)
    term2 = np.identity(n_feat) - mu * np.matmul(y_k, s.T)

    B = np.matmul(term1, np.matmul(B ,term2))  +  mu * np.matmul(s,s.T)
    
    k = k + 1
    x_k.append(x)
    

  return x, k, x_k



$\textbf{Solving by Newton's method with $\lambda$ = 0 (without regularization) and with $\lambda$ = 0.1 }$

In [ ]:
digits = load_digits()

print(digits.data.shape)
print(digits.target.shape)

N = digits.data.shape[0]
d = digits.data.shape[1]
A = digits.data
y = 1.0 * np.ones([A.shape[0],1])

for i in range(digits.target.shape[0]):
  y[i] = digits.target[i]



In [ ]:
my_x = np.zeros((A.shape[1],1))
lamb = [0, 0.1]
my_tol = 10e-5
alpha = 0.9
rho = 0.5
gamma = 0.5

try:
  x_0 , k_0 , x_k0 = find_minimizer_Newtonmethod(A,my_x,y,d,N,lamb[0],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
except Exception as e:
  print('Error : ',e)

x_1 , k_1 , x_k1 = find_minimizer_Newtonmethod(A,my_x,y,d,N,lamb[1],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)

print('Results :-')
print('---')

print('With $\lambda$ =0 , ERROR : Singular matrix \n ')
print('With $\lambda$ =0.1 , x_opt = ',x_1.T , '\n\n and the number of iterations taken is,' , k_1)

Error :  Singular matrix
Results :-
---
With $\lambda$ =0 , ERROR : Singular matrix 
 
With $\lambda$ =0.1 , x_opt =  [[ 0.00000000e+00  9.72176393e-02 -4.25221013e-03 -7.65725749e-03
   7.49359298e-02  1.13924666e-02 -2.68134811e-02 -8.48370171e-03
   9.91208545e-01 -2.87397984e-02  1.18690196e-01  6.61518400e-02
  -5.57615717e-02 -6.96340237e-02  9.62813519e-02  2.56470858e-01
  -7.28979627e-01  2.42825856e-02  7.72526071e-02 -2.33770172e-02
  -5.63320407e-02  5.71246069e-02 -4.84767009e-02 -2.70744170e-01
  -8.60889238e-01 -1.49941949e-01  5.64334649e-02  8.96806467e-02
   8.39114973e-02  9.85243348e-02  1.64759992e-03 -2.82145749e+00
   0.00000000e+00 -1.54275472e-01 -9.36618641e-03  1.39528972e-01
  -3.69438111e-02  5.46098301e-02 -9.13188784e-03  0.00000000e+00
   1.07369006e-01  1.23996365e-01 -1.37231270e-02  5.34871565e-03
   1.31237767e-01  5.50202750e-02  2.24738205e-02  7.53480641e-03
   5.95009063e-01  2.42332551e-02  1.44538782e-03 -6.21495531e-02
  -2.06985396e-01 -3.389

$\textbf{Solving by BFGS method with $\lambda$ = 0 (without regularization) and with $\lambda$ = 0.1 }$

In [ ]:
x_0 , k_0 , x_k0 = find_minimizer_BFGS(A,my_x,y,d,N,lamb[1],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
x_1 , k_1 , x_k1 = find_minimizer_BFGS(A,my_x,y,d,N,lamb[1],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)

print('For the BFGS Methods Without regualrization')
print('\n')
print('Optimal x : ', x_0.T)
print('Number of itertion : ', k_0)

print('--------')

print('For the BFGS Methods With lambda = 0.1')
print('\n')
print('Optimal x : ', x_1.T)
print('Number of itertion : ', k_1)


For the BFGS Methods Without regualrization


Optimal x :  [[ 0.00000000e+00  9.72176390e-02 -4.25220995e-03 -7.65725748e-03
   7.49359297e-02  1.13924666e-02 -2.68134811e-02 -8.48370168e-03
   9.91208546e-01 -2.87397983e-02  1.18690196e-01  6.61518400e-02
  -5.57615717e-02 -6.96340237e-02  9.62813520e-02  2.56470858e-01
  -7.28979638e-01  2.42825855e-02  7.72526070e-02 -2.33770172e-02
  -5.63320407e-02  5.71246069e-02 -4.84767009e-02 -2.70744170e-01
  -8.60889213e-01 -1.49941949e-01  5.64334649e-02  8.96806467e-02
   8.39114973e-02  9.85243348e-02  1.64759987e-03 -2.82145750e+00
   0.00000000e+00 -1.54275472e-01 -9.36618643e-03  1.39528972e-01
  -3.69438111e-02  5.46098302e-02 -9.13188786e-03  0.00000000e+00
   1.07369006e-01  1.23996365e-01 -1.37231270e-02  5.34871566e-03
   1.31237767e-01  5.50202749e-02  2.24738204e-02  7.53480609e-03
   5.95009064e-01  2.42332550e-02  1.44538782e-03 -6.21495531e-02
  -2.06985396e-01 -3.38924139e-02  1.05491772e-01 -1.40375388e-01
  -8.25290577e-01

***Remarks :***  *We observe that for the newtons's method we get a singular hessian matrix without regularization and hence we cannnot calculate the optimal value of x. But we get the optimal value of x when we use newton's method with regularization.*

*We observe that for the BFGS methods, we get the answer in both cases. i.e with and without regualrization.*